In [1]:
#import numpy as np
#import matplotlib.pyplot as plt
import corner
import emcee
from invisoplanet_detection.statistics import *

To run MCMC I need the following function call with arguments:
 - num_walker: number of walker to sample the distribution
 - num_dim: number of dimensions = number of parameters in the distribution. Here num_dim = number unknown planet (+ eta?)
 - log_post: name of callable function for posterior distribution. Must take num_dim parameters then take args
 - args: arguments passed to log_post
 - initial_pos: array of initial position for all the walkers. ex: initial_pos = np.array((2.6, 1.3)) + 0.01 * np.random.randn(nwalkers, ndim)
 - num_iter: number of iterations of the algorithm.

sampler = emcee.EnsembleSampler(num_walkers, num_dim, log_post, args=(x_vals, y_vals, errs))

sampler.run_mcmc(initial_pos, num_iter, progress=True);

I can process the data using the function calls:

samples = sampler.get_chain()

flat_samples = sampler.get_chain(discard=100, thin=15, flat=True)

Finally make the corner plot using:

fig = corner.corner(flat_samples, labels=labels, quantiles=[0.16, 0.5, 0.84]);

In [ ]:
#To build the likelihood function on each system use the following code example:

# Setup the likelihood object
known_bodies = 1
unknown_bodies = 2
parameters_filename = "invisoplanet_detection/data/sat_sun_jup_sat_1_2_2.json"
max_masses = np.array([1, 9.547919e-4]) * 2  # Actual masses times 2. Consider loading file then multiplying by 2?
surrogate_points = 9

# Optional arguments (these are the default values)
num_iterations = 20_000     
time_step = 0.5
last_n = 100        

# Construct the likelihood object
likelihood = Likelihood(known_bodies, unknown_bodies, parameters_filename, max_masses, surrogate_points,
						num_iterations, time_step, last_n)

# Set the eta value (the only parameter you can change without having to regenerate the surrogate model)
likelihood.set_eta(1)

#log of posterior distribution accessed from:
log_post = likelihood.log_posterior(guess_masses, x = None, y = None, yerr = None) 